In [1]:
import sqlite3
import csv
import pandas as pd
import numpy as np

import timeit

## Data load
### Pandas

In [3]:
csv_file = 'parking_citations.corrupted.csv'
# csv_file = 'shrunk_parking.csv'

In [5]:
pd_load_setup = "import pandas as pd"

pd_load_stmt = "tkt_data = pd.read_csv('" + csv_file + "', header=0, \
names=['tkt_num', 'tkt_dt', 'tkt_time', 'meter_id', 'marked_time', \
'state', 'plate_expr_dt', 'vin', 'make', 'body_style', \
'color', 'location','route', 'agency', 'viol_code', \
'viol_desc', 'fine_amt', 'latitude', 'longitude'], \
dtype={'tkt_num': object, 'vin': object})"

pd_load_time = timeit.timeit(pd_load_stmt, pd_load_setup, number=1)
print(pd_load_time)

exec(pd_load_stmt)
print(tkt_data.shape)
# The instructions say, "Do this analysis on the uncorrupted component of the data only."
tkt_data = tkt_data[tkt_data['make'].notnull()]
print(tkt_data.shape)
print(tkt_data.head())

50.69512118
(8726014, 19)
(4357544, 19)
       tkt_num               tkt_dt  tkt_time meter_id  marked_time state  \
4   1105461453  2015-09-15T00:00:00     115.0      NaN          NaN    CA   
6   1106500452  2015-12-17T00:00:00    1710.0      NaN          NaN    CA   
10  1106506424  2015-12-22T00:00:00    1100.0      NaN          NaN    CA   
11  1106506435  2015-12-22T00:00:00    1105.0      NaN          NaN    CA   
14  1107179581  2015-12-27T00:00:00    1055.0      NaN          NaN    CA   

    plate_expr_dt  vin  make body_style color              location  route  \
4        200316.0  NaN  CHEV         PA    BK    GEORGIA ST/OLYMPIC  1FB70   
6        201605.0  NaN  MAZD         PA    BL       SUNSET/ALVARADO  00217   
10       201511.0  NaN  FORD         TR    WH       1159 HUNTLEY DR   2A75   
11       201701.0  NaN  CHRY         PA    GO       1159 HUNTLEY DR   2A75   
14       201605.0  NaN  TOYO         PA    BK  3100 N HOLLYRIDGE DR    NaN   

    agency viol_code        

### SQLite

In [6]:
sqlite_insert_setup = """
import csv, sqlite3

csv_file = 'parking_citations.corrupted.csv'
# csv_file = 'shrunk_parking.csv'

conn = sqlite3.connect("tkt_db.db")
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS tkt_data')
conn.commit()
c.execute('CREATE TABLE IF NOT EXISTS tkt_data \
(tkt_num text, \
tkt_dt text, \
tkt_time real, \
meter_id text, \
marked_time text, \
state text, \
plate_expr_dt real, \
vin text, \
make text, \
body_style text, \
color text, \
location text, \
route text, \
agency text, \
viol_code text, \
viol_desc text, \
fine_amt real, \
latitude real, \
longitude real)')
conn.commit()
"""

sqlite_insert_stmt = """
with open(csv_file, 'r') as csvfile: 
    row = csv.reader(csvfile) 
    next(row) # skip header row
    c.executemany('INSERT INTO tkt_data VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);', row)
conn.commit()
"""

sqlite_insert_time = timeit.timeit(sqlite_insert_stmt, sqlite_insert_setup, number=1)
print(sqlite_insert_time)

81.76237056699998


#### For both ease of coding and load time, Pandas is the clear winner for loading data.
#### Pandas took one line of code, while SQLite required several.
#### Locally, for me, Pandas loads this dataset 30 - 50 seconds faster than SQLite.

In [7]:
conn = sqlite3.connect("tkt_db.db")
c = conn.cursor()
c.execute("SELECT count(*) FROM tkt_data")
print(c.fetchone())
# The instructions say, "Do this analysis on the uncorrupted component of the data only."
c.execute('DELETE FROM tkt_data WHERE make = ""')
conn.commit()
c.execute("SELECT count(*) FROM tkt_data")
print(c.fetchone())
c.execute("SELECT * FROM tkt_data")
print(c.fetchone())

(8726014,)
(4357544,)
('1105461453', '2015-09-15T00:00:00', 115.0, '', '', 'CA', 200316.0, '', 'CHEV', 'PA', 'BK', 'GEORGIA ST/OLYMPIC', '1FB70', '1.0', '8069A', 'NO STOPPING/STANDING', 93.0, 99999.0, 99999.0)


## a) calculate top 25 most common 'makes'
### Pandas

In [8]:
pd_a_setup = """import pandas as pd
""" + pd_load_stmt

pd_a_stmt = "pd_a = tkt_data.groupby(['make']).size().sort_values(ascending=False)"

pd_a_time = timeit.timeit(pd_a_stmt, pd_a_setup, number=1)
print(pd_a_time)

exec(pd_a_stmt)
print(pd_a.head(5))

1.222612220999963
make
TOYT    721411
HOND    491961
FORD    382695
NISS    311324
CHEV    297076
dtype: int64


#### Consistency in abbreviation is a problem with the Make data. Within the top 25 above, "TOYT" and "TOYO" almost certainly both are intended to represent Toyota. This is just an example of a broad issue.
#### To attempt to minimize this issue, Edit Distance is used to match the most common entries with less common entries to combine their totals. This creates its own problems and questions though.
####  
#### TOYT == TOYO ?     Yes, both are Toyota.
#### VOLV == VOLK ?      No, Volvo and Volkswagen.
#### MERZ == MERC ?    Maybe. MERZ is probably Mercedes Benz. MERC could also mean a Benz, or maybe a Mercury.

In [9]:
# use edit distance to improve Make consistency

import nltk
 
makes = pd_a.to_frame()
makes.reset_index(inplace=True)
makes.rename(columns={makes.columns[1]:'count'}, inplace=True)
makes_dict = makes.to_dict()

len(makes)
print(makes.iloc[22,0])

makes['rplc_make'] = makes['make']
for i in range(len(makes.head(100))):
    for j in range(len(makes)):
        if (makes.iloc[i,1] >= makes.iloc[j,1] 
           and nltk.edit_distance(makes.iloc[i,0],makes.iloc[j,0]) == 1
           and makes['make'].iloc[j] not in ['VOLV', 'MERC', 'BENZ']
           ): # and makes.iloc[j,1]>99):
#             print(makes.iloc[i,0] +':'+ str(makes.iloc[i,1]) +', ' \
#                   + makes.iloc[j,0] +':'+ str(makes.iloc[j,1]) +', ' \
#                   + str(nltk.edit_distance(makes.iloc[i,0],makes.iloc[j,0])) + ', ' + str(i))
            makes['rplc_make'].iloc[j] = makes.iloc[i,0]
            break


TOYO


/Users/user/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [11]:
(makes[makes['make'] != makes['rplc_make']]).head(20)

,make,count,rplc_make
22,TOYO,40064,TOYT
45,LEXU,5098,LEXS
60,MINI,1170,MNNI
64,CHEC,871,CHEV
73,LEX,429,LEXU
74,SUZU,381,SUZI
75,ISUZ,376,ISU
76,JAGR,367,JAGU
77,SAAB,347,SAA
79,LROV,326,RROV


#### This is getting too bespoke and not really helping. With more time, an improved strategy could be implemented.

### SQLite

In [12]:
a_query_setup = """
import sqlite3
con = sqlite3.connect('tkt_db.db')
c = con.cursor()
"""

a_query_stmt = """c.execute('SELECT count(*), make \
FROM tkt_data \
WHERE make != "" \
GROUP BY make \
ORDER BY count(*) DESC')"""

in_num = 1
a_time = timeit.timeit(a_query_stmt, a_query_setup, number=in_num)
print(a_time)

exec(a_query_stmt)
print(c.fetchmany(25))

45.49994978899997
[(721411, 'TOYT'), (491961, 'HOND'), (382695, 'FORD'), (311324, 'NISS'), (297076, 'CHEV'), (199221, 'BMW'), (177307, 'MERZ'), (149501, 'VOLK'), (133864, 'HYUN'), (127764, 'DODG'), (124508, 'LEXS'), (101746, 'KIA'), (100909, 'JEEP'), (84229, 'AUDI'), (79853, 'MAZD'), (72411, 'OTHR'), (62391, 'GMC'), (57317, 'CHRY'), (56809, 'INFI'), (52703, 'ACUR'), (46898, 'SUBA'), (42330, 'VOLV'), (40064, 'TOYO'), (37842, 'MITS'), (34080, 'CADI')]


#### Pandas wins again: faster with similar code quantity.

## b) calculate most common 'Color' for each 'Make'
### Pandas

In [13]:
pd_b_stmt = """b_df = tkt_data.groupby(['make','color']).size().to_frame().reset_index()
b_df.rename(columns={b_df.columns[2]:'count'}, inplace=True)
b_max = b_df[b_df.groupby(['make'])['count'].transform(max) == b_df['count']].sort_values('count', ascending=False)"""

b_time = timeit.timeit(pd_b_stmt, pd_a_setup, number=1)
print(b_time)

exec(pd_b_stmt)
print(b_max.head())

1.9046794160000218
      make color   count
4667  TOYT    GY  163238
1635  HOND    GY  116566
1255  FORD    WT  112313
648   CHEV    WT   78792
3312  NISS    GY   72622


In [14]:
b_query_stmt = """c.execute(' \
    SELECT make, color, ct \
    FROM \
      (SELECT make, color, count(*) as ct \
       FROM tkt_data \
       WHERE make != "" \
       GROUP BY make, color) \
    GROUP BY make \
    HAVING ct=max(ct) \
    ORDER BY ct DESC')"""

b_query_time = timeit.timeit(b_query_stmt, a_query_setup, number=1)
print(b_query_time)

exec(b_query_stmt)
print(c.fetchmany(5))

41.10396065300006
[('TOYT', 'GY', 163238), ('HOND', 'GY', 116566), ('FORD', 'WT', 112313), ('CHEV', 'WT', 78792), ('NISS', 'GY', 72622)]


#### Pandas is definitely faster, but the SQL code for this seems more straight forward. Call it a split decision that leans towards Pandas.

## c) find the first ticket issued for each 'Make'
### Pandas

In [15]:
pd_c_stmt = "c_df = tkt_data.groupby(['make'])['tkt_dt','tkt_num'].min(by='tkt_dt').sort_values('tkt_dt')"

pd_c_time = timeit.timeit(pd_c_stmt, pd_a_setup, number=1)
print(pd_c_time)

exec(pd_c_stmt)
print(c_df.head(5))

7.222566290000032
                   tkt_dt     tkt_num
make                                 
FORD  2010-01-09T00:00:00  1001096471
ACUR  2010-06-09T00:00:00  1036519013
KIA   2010-06-10T00:00:00  1003412955
CHEV  2010-06-25T00:00:00  1001096331
JEEP  2010-10-05T00:00:00  1018619932


### SQLite

In [17]:
c_query_stmt = """c.execute(" \
SELECT min(tkt_dt) as min_tkt_dt, make, tkt_num \
FROM tkt_data \
WHERE make != '' \
GROUP BY make \
ORDER BY min_tkt_dt")"""

c_query_time = timeit.timeit(c_query_stmt, a_query_setup, number=1)
print(c_query_time)

exec(c_query_stmt)
print(c.fetchmany(2))

5.437045543999943
[('2010-01-09T00:00:00', 'FORD', '1108852275'), ('2010-06-09T00:00:00', 'ACUR', '1112627854')]


#### Another tough call as far as code, but again, Pandas is faster. Pandas wins.

### Is an out-of-state license plate more likely to be expired at the time of receiving a ticket than an in-state license plate?

In [84]:
tkt_data.groupby(['state']).size().sort_values(ascending=False).head()

state
CA    4058883
AZ      38914
TX      27415
NV      26067
FL      19105
dtype: int64

In [86]:
tkt_data.location[910800:910802]

1823264            707 LUCILE AVE
1823269    APPX 2742 GLENN AVENUE
Name: location, dtype: object

#### There are more than 100x as many instances of California plates than any other single state. Every Location I've looked up is in Los Angeles. Therefore, I am assuming all of the tickets in the dataset were given in California. So any non-"CA" plate is out-of-state.

In [87]:
in_out_expr = tkt_data[['tkt_dt','state','plate_expr_dt']].dropna()
in_out_expr.head()

,tkt_dt,state,plate_expr_dt
4,2015-09-15T00:00:00,CA,200316.0
6,2015-12-17T00:00:00,CA,201605.0
10,2015-12-22T00:00:00,CA,201511.0
11,2015-12-22T00:00:00,CA,201701.0
14,2015-12-27T00:00:00,CA,201605.0


In [88]:
in_out_expr.groupby(['state']).size().sort_values(ascending=False).head()

state
CA    3723823
NV      24378
TX      21694
AZ      18950
FL      18367
dtype: int64

In [89]:
in_out_expr['tkt_year'] = in_out_expr.tkt_dt.str.slice(0,4).astype(float)
in_out_expr['tkt_month'] = in_out_expr.tkt_dt.str.slice(5,7).astype(float)
in_out_expr['plate_year'] = (in_out_expr.plate_expr_dt - (in_out_expr.plate_expr_dt % 100)) / 100
in_out_expr['plate_month'] = in_out_expr.plate_expr_dt % 100
in_out_expr.head()

,tkt_dt,state,plate_expr_dt,tkt_year,tkt_month,plate_year,plate_month
4,2015-09-15T00:00:00,CA,200316.0,2015.0,9.0,2003.0,16.0
6,2015-12-17T00:00:00,CA,201605.0,2015.0,12.0,2016.0,5.0
10,2015-12-22T00:00:00,CA,201511.0,2015.0,12.0,2015.0,11.0
11,2015-12-22T00:00:00,CA,201701.0,2015.0,12.0,2017.0,1.0
14,2015-12-27T00:00:00,CA,201605.0,2015.0,12.0,2016.0,5.0


In [90]:
in_out_expr.groupby(['state']).size().sort_values(ascending=False).head()

state
CA    3723823
NV      24378
TX      21694
AZ      18950
FL      18367
dtype: int64

In [91]:
in_out_expr.groupby(['plate_month']).size().sort_values(ascending=False)

plate_month
6.0     352190
9.0     347028
8.0     347004
3.0     344247
5.0     340262
7.0     336193
4.0     333975
1.0     330400
2.0     315740
12.0    309022
11.0    303308
10.0    297733
17.0       826
18.0       775
16.0       671
0.0        607
15.0       586
19.0       288
14.0       109
13.0        22
20.0         4
98.0         2
60.0         1
21.0         1
24.0         1
30.0         1
36.0         1
dtype: int64

#### Plate expiration months greater outside of 1-12 occur rarely enough to be considered negligible. Those where the year is not enough to determine if the plate was expired will be dropped.

In [110]:
# in_out_expr_bkup = in_out_expr.copy
in_out_expr['expired'] = False

in_out_expr.loc[in_out_expr['plate_year'] < in_out_expr['tkt_year'], 'expired'] = True
print(len(in_out_expr))
in_out_expr.drop(in_out_expr[((in_out_expr['plate_month'] < 1) | (in_out_expr['plate_month'] > 12)) 
            & (in_out_expr['plate_year'] == in_out_expr['tkt_year'])].index, inplace=True)
print(len(in_out_expr))

in_out_expr.loc[(in_out_expr['plate_year'] == in_out_expr['tkt_year'])
                & (in_out_expr['plate_month'] < in_out_expr['tkt_month']), 'expired'] = True

in_out_expr.head(10)

3960575
3960575


,tkt_dt,state,plate_expr_dt,tkt_year,tkt_month,plate_year,plate_month,expired,in_state
4,2015-09-15T00:00:00,CA,200316.0,2015.0,9.0,2003.0,16.0,True,True
6,2015-12-17T00:00:00,CA,201605.0,2015.0,12.0,2016.0,5.0,False,True
10,2015-12-22T00:00:00,CA,201511.0,2015.0,12.0,2015.0,11.0,True,True
11,2015-12-22T00:00:00,CA,201701.0,2015.0,12.0,2017.0,1.0,False,True
14,2015-12-27T00:00:00,CA,201605.0,2015.0,12.0,2016.0,5.0,False,True
19,2015-12-22T00:00:00,CA,201606.0,2015.0,12.0,2016.0,6.0,False,True
21,2015-12-18T00:00:00,CA,201509.0,2015.0,12.0,2015.0,9.0,True,True
22,2015-12-18T00:00:00,CA,201511.0,2015.0,12.0,2015.0,11.0,True,True
25,2015-12-22T00:00:00,CA,201610.0,2015.0,12.0,2016.0,10.0,False,True
29,2015-12-22T00:00:00,CA,201601.0,2015.0,12.0,2016.0,1.0,False,True


In [111]:
in_out_expr['in_state'] = in_out_expr['state']=='CA'
in_out_expr.head()

,tkt_dt,state,plate_expr_dt,tkt_year,tkt_month,plate_year,plate_month,expired,in_state
4,2015-09-15T00:00:00,CA,200316.0,2015.0,9.0,2003.0,16.0,True,True
6,2015-12-17T00:00:00,CA,201605.0,2015.0,12.0,2016.0,5.0,False,True
10,2015-12-22T00:00:00,CA,201511.0,2015.0,12.0,2015.0,11.0,True,True
11,2015-12-22T00:00:00,CA,201701.0,2015.0,12.0,2017.0,1.0,False,True
14,2015-12-27T00:00:00,CA,201605.0,2015.0,12.0,2016.0,5.0,False,True


In [109]:
print('Probability out of state plates are expired: ' + 
      str(round(float(len(in_out_expr[(in_out_expr['expired'] == 'YES') 
                                      & (in_out_expr['state'] != 'CA')]))/ \
len(in_out_expr[in_out_expr['state'] != 'CA']),4)))

print('Probability in state plates are expired: ' + 
      str(round(float(len(in_out_expr[(in_out_expr['expired'] == 'YES') 
                                      & (in_out_expr['state'] == 'CA')]))/ \
len(in_out_expr[in_out_expr['state'] == 'CA']),4)))

Probability out of state plates are expired: 0.2434
Probability in state plates are expired: 0.233


In [118]:
from scipy.stats import ttest_ind

stat, p = ttest_ind(in_out_expr.loc[in_out_expr['state']=='CA', 'expired'],
                    in_out_expr.loc[in_out_expr['state']!='CA', 'expired'])

print(p)

2.239748891899383e-31


#### The difference between the two distributions is significant, so my final answer is "Yes, out of state plates are more likely to be expired than in state plates when a ticket is given."